In [2]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [3]:
inter_df = pd.read_csv('./국토교통부_건축물대장_총괄표제부/wbdinfo_baseline_maindata_inter_checpoint_1.csv')
print(inter_df.shape)
inter_df.head()

(45376, 43)


c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년도_최초사용승인일,기준년도_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
0,11650-20602,집합,총괄표제부,구대장,0,359.3,162.78,45.30,493.41,325.56,90.61,공동주택,공동주택,2,0,0.0,2,0,0.0,0,0.0,0,0.00,2,30.00,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 911-29,8,일반,NaN,NaN,20100417,20171118,8
1,11650-21702,집합,총괄표제부,신대장,0,4321.4,1855.67,42.94,12660.24,6129.15,141.83,공동주택,공동주택(연립주택),3,0,0.0,0,0,0.0,0,0.0,110,4522.93,0,0.00,20030416.0,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 서초동 1496-26,18,일반,20030416.0,20030416.0,20100417,20161220,18
2,11650-39702,집합,총괄표제부,신대장,1,884.0,413.10,46.73,3695.03,2346.09,265.39,공동주택,공동주택(아파트),2,0,0.0,28,0,0.0,0,0.0,28,1305.56,0,0.00,20030514.0,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 1-7,16,일반,20030514.0,20030514.0,20100417,20171118,16
3,11650-41002,집합,총괄표제부,구대장,1,380.5,190.20,49.98,570.60,380.40,99.97,공동주택,공동주택,2,0,0.0,3,0,0.0,0,0.0,0,0.00,3,41.25,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 반포동 577-73,12,일반,NaN,NaN,20100417,20100417,12
4,11650-4302,집합,총괄표제부,신대장,0,493.0,292.30,59.29,1234.89,1095.26,222.16,공동주택,"공동주택,다세대주택",2,0,0.0,12,0,0.0,0,0.0,12,0.00,0,0.00,20030925.0,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 919-10,15,일반,20030925.0,20030925.0,20100417,20171118,15


In [4]:
total_house_number_df = inter_df[['지번주소', '기준년월', '관리_건축물대장_PK', '주소별_기준년월_총세대수합',
                                  '기준년도_최초사용승인일', '주소별_최초생성일자']].drop_duplicates().sort_values(['지번주소', '기준년월'])
print(total_house_number_df.shape)
total_house_number_df.head()

(45376, 6)


,지번주소,기준년월,관리_건축물대장_PK,주소별_기준년월_총세대수합,기준년도_최초사용승인일,주소별_최초생성일자
28371,서울특별시 강남구 개포동 1193-5,201812,11680-100276255,29,20181112.0,20181114
35072,서울특별시 강남구 개포동 1193-5,201912,11680-100276255,29,20181112.0,20181114
41096,서울특별시 강남구 개포동 1193-5,202011,11680-100276255,29,20181112.0,20181114
3306,서울특별시 강남구 개포동 12-0,201412,11680-10302,4199,NaN,20120822
11851,서울특별시 강남구 개포동 12-0,201512,11680-103,4199,NaN,20120822


In [5]:
inv_df = total_house_number_df[total_house_number_df['지번주소'] == '서울특별시 강남구 개포동 12-0']
inv_df

,지번주소,기준년월,관리_건축물대장_PK,주소별_기준년월_총세대수합,기준년도_최초사용승인일,주소별_최초생성일자
3306,서울특별시 강남구 개포동 12-0,201412,11680-10302,4199,NaN,20120822
11851,서울특별시 강남구 개포동 12-0,201512,11680-103,4199,NaN,20120822
16539,서울특별시 강남구 개포동 12-0,201612,11680-103,4199,NaN,20120822
18813,서울특별시 강남구 개포동 12-0,201712,11680-103,4199,NaN,20120822
28652,서울특별시 강남구 개포동 12-0,201812,11680-103,4199,NaN,20120822
35391,서울특별시 강남구 개포동 12-0,201912,11680-103,4199,NaN,20120822
39048,서울특별시 강남구 개포동 12-0,202011,11680-103,4199,NaN,20120822


In [6]:
total_house_number_df.to_csv('./prepped_data/총괄표제부_기준년월별_총세대수합.csv', index=False)